In [1]:
import pathlib
import pandas as pd
import shutil
import logging

from idrstream.idr_stream import IdrStream

In [2]:
data_to_process = pd.read_csv("example_files/data_to_process.tsv", sep="\t", index_col=0)
# remove A1 wells because of irregular illumination
data_to_process = data_to_process[data_to_process["Well"] != "A1"]
data_to_process = data_to_process.reset_index(drop=True)
data_to_process

,Plate,Well,Frames,Well Number,Control Type,Original Gene Target
0,LT0001_02,A4,50,4,positive control,ENSG00000149503
1,LT0001_02,A15,50,15,negative control,negative control
2,LT0001_02,B2,50,26,negative control,negative control
3,LT0001_02,C1,50,49,positive control,ENSG00000149503
4,LT0001_02,C4,50,52,positive control,ENSG00000149503
...,...,...,...,...,...,...
6738,LT0603_06,O13,50,349,positive control,KIF11
6739,LT0603_06,O16,50,352,negative control,negative control
6740,LT0603_06,P3,50,363,negative control,negative control
6741,LT0603_06,P21,50,381,positive control,COPB


In [3]:
idr_id = "idr0013"
tmp_dir = pathlib.Path("tmp/")
final_data_dir = pathlib.Path("mitocheck_control_features/")
try:
    shutil.rmtree(tmp_dir)
    # shutil.rmtree(final_data_dir)
    pass
except:
    print("No files to remove")

stream = IdrStream(idr_id, tmp_dir, final_data_dir, log='idr_stream.log')

No files to remove


In [4]:
aspera_path = pathlib.Path("/home/roshankern/.aspera/ascli/sdk/ascp")
aspera_key_path = pathlib.Path("example_files/asperaweb_id_dsa.openssh")
screens_path = pathlib.Path("example_files/idr0013-screenA-plates.tsv")
save_dir = pathlib.Path("data/")

stream.init_downloader(aspera_path, aspera_key_path, screens_path)

In [5]:
fiji_path = pathlib.Path("/home/roshankern/Desktop/Github/IDR_stream_prototype/preprocess_data/Fiji.app")
stream.init_preprocessor(fiji_path)

[INFO] Overriding Leica ROI Reader; identifier: command:de.biovoxxel.utilities.RoiReader; jar: file:/home/roshankern/Desktop/Github/IDR_stream_prototype/preprocess_data/Fiji.app/plugins/Biovoxxel_Plugins-2.5.6.jar


[ERROR] Cannot create plugin: org.scijava.plugins.scripting.javascript.JavaScriptScriptLanguage


In [6]:
nuclei_model_specs = {
            "model_type": "cyto",
            "channels": [0, 0],
            "diameter": 0,
            "flow_threshold": 0.8,
            "cellprob_threshold": 0,
            "remove_edge_masks": True,
        }
stream.init_segmentor(nuclei_model_specs)

>>> GPU activated? 1


In [7]:
config_path = pathlib.Path("example_files/DP_files/mitocheck_profiling_config.json")
checkpoint_path = pathlib.Path("example_files/DP_files/efficientnet-b0_weights_tf_dim_ordering_tf_kernels_autoaugment.h5")
stream.copy_DP_files(config_path, checkpoint_path)

In [ ]:
stream.run_stream(data_to_process, batch_size=3, start_batch=0)